In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

print("MPS Available:", torch.backends.mps.is_available())
# Set up device for MPS
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")


# Step 1: Data Preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1000, shuffle=False)

# Step 2: Define the Model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.model(x)



In [ ]:
def train_model(model, train_loader, optimizer, criterion, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for data, target in train_loader:
            data, target = data.to(device), target.to(device) # Move computations to MPS (GPU)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


In [ ]:
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device) # Move computations to MPS (GPU)
            output = model(data)
            pred = output.argmax(dim=1)
            correct += (pred == target).sum().item()
            total += target.size(0)
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy * 100:.2f}%")
    return accuracy


In [ ]:
learning_rates = [0.001, 0.01, 0.1]
epoch_list = [1, 3, 5, 10]

results = []
for lr in learning_rates:
    for num_epochs in epoch_list:
        model = SimpleNN().to(device) # Move model to MPS (GPU)
        optimizer = optim.SGD(model.parameters(), lr=lr)
        criterion = nn.CrossEntropyLoss()
        print(f"\nTraining with lr={lr}, epochs={num_epochs}")
        train_model(model, train_loader, optimizer, criterion, num_epochs)
        acc = test_model(model, test_loader)
        results.append((lr, num_epochs, acc))


In [ ]:
import matplotlib.pyplot as plt

# Group and plot results
for lr in learning_rates:
    xs = []
    ys = []
    for r in results:
        if r[0] == lr:
            xs.append(r[1])  # epochs
            ys.append(r[2])  # accuracy
    plt.plot(xs, ys, label=f"lr={lr}")

plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy vs Epochs for different Learning Rates")
plt.legend()
plt.grid(True)
plt.show()
